In [1]:
#@title Initial setup
from typing import Optional
import warnings
# Disable annoying warnings from PyArrow using under the hood.
warnings.simplefilter(action='ignore', category=FutureWarning)


import tensorflow as tf
import dask.dataframe as dd
from waymo_open_dataset import v2
from waymo_open_dataset.utils import  frame_utils
print(tf.__version__)


# Path to the directory with all components
dataset_dir = '/media/mike/MainDrive/Documents/data/perception_2.0.0/training'

context_name = '10023947602400723454_1120_000_1140_000'

def read(tag: str) -> dd.DataFrame:
  """Creates a Dask DataFrame for the component specified by its tag."""
  paths = tf.io.gfile.glob(f'{dataset_dir}/{tag}/{context_name}.parquet')
  return dd.read_parquet(paths)


2024-06-26 19:21:33.571721: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-26 19:21:33.780767: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-26 19:21:33.782695: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-26 19:21:34.699758: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


2.12.0


In [2]:

# Lazily read DataFrames for all components.
association_df = read('camera_to_lidar_box_association')
cam_box_df = read('camera_box')
cam_img_df = read('camera_image')
lidar_box_df = read('lidar_box')
lidar_df = read('lidar')
lidar_calibration_df = read('lidar_calibration')

# Join all DataFrames using matching columns
cam_image_w_box_df = v2.merge(cam_box_df, cam_img_df)
cam_obj_df = v2.merge(association_df, cam_image_w_box_df)
# In this example camera box labels are optional, so we set left_nullable=True.
obj_df = v2.merge(cam_obj_df, lidar_box_df, left_nullable=True)
# Group lidar sensors (left), group labels and camera images (right) and join.
df = v2.merge(lidar_df, obj_df, left_group=True, right_group=True)

# Read a single row, which contain data for all data for a single frame.
_, row = next(iter(df.iterrows()))
# Create all component objects
camera_image = v2.CameraImageComponent.from_dict(row)
lidar = v2.LiDARComponent.from_dict(row)
camera_box = v2.CameraBoxComponent.from_dict(row)
lidar_box = v2.LiDARBoxComponent.from_dict(row)
lidar_calibration = v2.LiDARCalibrationComponent.from_dict(lidar_calibration_df)

print(
    f'Found {len(lidar_box.key.laser_object_id)} objects on'
    f' {lidar.key.segment_context_name=} {lidar.key.frame_timestamp_micros=}'
)
for laser_object_id, camera_object_id, camera_name in zip(
    lidar_box.key.laser_object_id,
    camera_box.key.camera_object_id,
    camera_image.key.camera_name,
):
  print(f'\t{laser_object_id=} {camera_object_id=} {camera_name=}')

Found 81 objects on lidar.key.segment_context_name='10023947602400723454_1120_000_1140_000' lidar.key.frame_timestamp_micros=1552440195362591
	laser_object_id='-U88NMYnocLWCh6iqZwj1g' camera_object_id=nan camera_name=nan
	laser_object_id='0VCoeT-jjrIfzTCsOWz20A' camera_object_id=nan camera_name=nan
	laser_object_id='63vjcxiQGxy6SmQ94yX-FQ' camera_object_id=nan camera_name=nan
	laser_object_id='81faaqjwezt3wc0ZN-1N0g' camera_object_id=nan camera_name=nan
	laser_object_id='A4kzX4tBck4XcFuUOPA6Lg' camera_object_id='42d3fd29-cc31-44c6-a044-7af1a172488e' camera_name=3.0
	laser_object_id='CB379cbEIKwW6eAs4wK6mw' camera_object_id='baae00a2-cacd-43c9-9d83-0932920c987e' camera_name=1.0
	laser_object_id='Iv3M96stRJ3JE-umZR0uVA' camera_object_id=nan camera_name=nan
	laser_object_id='L5HBM5tSKtNA4qHegDQk8Q' camera_object_id='4b9a9206-fbf1-4d45-8765-197759100bd6' camera_name=1.0
	laser_object_id='PrS_b18HNPEbxJ9IAYW2BQ' camera_object_id=nan camera_name=nan
	laser_object_id='YTw2k0_-8UqolJn4WQRc2g' 

In [3]:
# Get Range image of top lidar
range_image = lidar.range_image_return1.values[0]
print(range_image[0])
print(lidar.range_image_return1.shape)

# TODO: Eventually should add pixel_pose and frame_pose when mulitple cameras are used
points = v2.perception.utils.lidar_utils.convert_range_image_to_point_cloud(range_image, lidar_calibration, keep_polar_features=True)


-1.0
[array([  64, 2650,    4], dtype=int32), array([200, 600,   4], dtype=int32), array([200, 600,   4], dtype=int32), array([200, 600,   4], dtype=int32), array([200, 600,   4], dtype=int32)]


AttributeError: 'numpy.ndarray' object has no attribute 'tensor'